In [1]:
import sys
import numpy 

print(sys.version_info)

sys.version_info(major=3, minor=8, micro=8, releaselevel='final', serial=0)


## Common Parameters and what they mean
G = specific heat ratio.  
RADB = OUTER RADIUS (in m, for calculations in real quantities)  
RADA = INNER RADIUS (in m,  ditto )  
Z0   = LENGTH OF ANODE (in m, ditto)   
C    = RADB/RADA.   
F    = Z0/RADA.   
L0   = CIRCUIT STRAY INDUCTANCE (in HENRY, ditto).  
C0   = ENERGY STORAGE CAPACITANCE (in FARAD, ditto).   
AL= capacitor time T0/TA, axial run-down time.  
BE= full axial phase inductance LZ0/L0.  
MASSF= REDUCED MASS FACTOR DUE TO MASS SHEDDING.   
CURRF= REDUCED CURRENT FACTOR DUE TO CURRENT SHEDDING.   
R0 IS STRAY CIRCUIT RESISTANCE IN OHM   
RESF=R0/Surge Impedance   
Z1 is end position, to end calculation of phase 5.  
MASSF, MASSFR are incorporated in TA & AL & in AA.  
CURRF IS INcorporated into BE.  
For Phase I calculations, T = Time, Z = Axial position (normalized to Zo).   
I = Current, ZZ = Speed, AC = Acceleration.  
II = Current derivative, I0 = Current Intergral, all normalized.   
HOWEVER ALL QUANTITIES WITH An R ATTACHED, HAVE BEEN RE-COMPUTED TO GIVE LABORATORY VALUES; e.g. TR IS TIME RE-COMPUTED IN MICROSEC;IR IS CURRENT RE-COMPUTED IN kA AND SO ON.  
D = Time increment, V = tube voltage, all normalized.   


In [20]:
#For now, default to UNUICTPPFF device 
L0 = 110
C0 = 30
RADB = 3.2
RADA = 0.95
Z0 = 5
R0 = 12
V0 = 15
P0 = 3.5
MW = 4
#atomic z number
ZN = 1
dissociatenumber = 2
massf = 0.08
CURRF = 0.7
massfr = 0.16
currfr = 0.7

R0 = R0 / 1000
#Setting consts
#magnetic permitivity of vaccum, Henry per meter 
MU = 1.257e-6
# \pi
Pi = 3.142
#boltzman const
bc = 1.38e-23
#mass of proton
mi = 1.67e-27
MUK = MU / (8*Pi*Pi*bc)
CON11 = 1.6e-20
CON12 = 9.999999e-21
CON2 = 4.6e-31
UGCONS = 8.310001e3
FRF = 0.3
fe = 1 / 3
FLAG = 0

#Rem reset EINP, energy dissipated by dynamic resistance effect, resistance for non-ohmic materials, generally r = dV/dI. which is 0.5 (Ldot) I^2, generally r =  considering current taking part in the motion

#gamma = specific heat ratio, was just 'g' in original vbs
gamma = 1.66667
#G1 is current sheet speed slower factor
G1 = 2/ (gamma + 1)
G2 = (gamma - 1) / gamma
#Compression ratio of ambient gas swept up by shock front 
GCAP = (gamma + 1) / (gamma - 1)

# Calculate ambient atomic number density (no of atoms per m^2)and some ratios
# For PF operation assume all molecules are completely dissociated in all phases

# N0= number of atoms per m^2; atomic number density
N0 = dissociatenumber * 2.69e25 * P0 / 760
# C is ratio of cathode radius / anode radius
C = RADB / RADA
# Lee paper, page 8. Z0 is the length of anode, so f is ratio of anode length over it's radius
f = Z0 / RADA
# Convert to SI units
C0 = C0 * pow(10,-6)
print(C0)
L0 = L0 * pow(10,-9)
RADB = RADB * 0.01
RADA = RADA * 0.01
Z0 = Z0 * pow(10,-2)
V0 = V0 * 1000
RHO = P0 * 2.33e-4 * MW / 4
TM = 0
#assuming no taper
#Calculate characteristic quantities and scaling parameters
#Lee paper page 5, Voltage at t = 0 over surge impedence
I0 = V0 / numpy.sqrt(L0 / C0)
#Lee paper page 5, 
T0 = numpy.sqrt(L0 * C0)
#Scaling parameter for Axel Phase. Equation 5 on Lee Paper
print(((Z0 * numpy.sqrt(RHO)) / (I0 / RADA)) * (numpy.sqrt(massf) / CURRF))
TA = numpy.sqrt(4 * Pi * Pi * (C * C - 1) / (MU * numpy.log(C))) * ((Z0 * numpy.sqrt(RHO)) / (I0 / RADA)) * (numpy.sqrt(massf) / CURRF)
#Lee papaer page 5, equation 7. zzchar is characteristic axial speed 
ZZCHAR = Z0 / TA
# AL : Alpha, ratio of characteristic electrical discharge time to the characteristic axial transit time
AL = T0 / TA
# Scaling Parameter for Radial Phase. Equation 24a, Lee paper page 8
AA = numpy.sqrt((gamma + 1) * (C * C - 1) / numpy.log(C)) * (f / 2) * (numpy.sqrt(massf / massfr)) * (currfr / CURRF)
# Ratio of stray resistance to surge impedence
RESF = R0 / (numpy.sqrt(L0 / C0))
#Lee paper page 5, equation 9. Ratio between inductance of the axial phase when CS reaches anode end z = z0, over Lo (inductance at t = 0) * current shedding factor, 
BE = 2e-7 * numpy.log(C) * Z0 * CURRF / L0
#Lee paper page 8, equation 24, Scaling parameter for Radial Inward Shock Phase
BF = BE / (numpy.log(C) * f)
#Lee paper page 8, equation 28, VPINCHCH is the speed of radial inward shock
VPINCHCH = ZZCHAR * AA / f
#convert speed of radial inward shock to time
TPINCHCH = RADA / VPINCHCH
#Calculate ratio of characteristic capacitor time to sum of characteristic axial & radial times
ALT = (AL * AA) / (1 + AA)

#Parameters sanity check
print(I0)
if ALT < 0.68:
    raise ValueError("Sorry ALT values are insane")





2.9999999999999997e-05
2.2125699104131396e-11
247716.84715343112


In [21]:
#Set time constants
d = 0.002
T = 0
z = 0
ZZ = 0
II = 1
I = 0
IO = 0
AC = AL * numpy.sqrt(1 / 2)





In [22]:
#Integration for Axial Phase
Ipeak = 0
Wpiston = 0
EINP = 0
V = 0
axialphase = []
# Start integration loop
while True:
    T = T + d
    if T > 6:
        raise ValueError("Sorry T is thru the roof!")
    ZZ = ZZ + AC * d
    z = z + ZZ * d
    I = I + II * d
    IO = IO + I * d
    #Init array-of-arrays for axial phase results
   

    #Unit conversion
    TR = T * T0* pow(10,6)
    VR = V * V0*pow(10,-3)
    IR = I * I0*pow(10,-3)
    ZZR = (ZZCHAR / AL) * ZZ * pow(10, -4)
    ACR = ((ZZCHAR / AL) / T0) * AC * pow(10, -10)
    ZR = z * Z0 * 100
    IIR = (II * I0 / T0) * pow(10, -9)
    IOR = IO * I0 * T0

    if IR > Ipeak:
        Ipeak = IR
    #DR is time increments in secs
    DR = d * T0
    #DZR is z increment in m
    DZR = DR * ZZR * pow(10, 4)
    #Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
    EINP = EINP + pow(10, -7) * (numpy.log(C) * ZZR * pow(10, 4) * IR * IR * pow(10 ,6) * CURRF * CURRF) * DR
    #integrate for piston work
    Wpiston = Wpiston + pow(10, -7) * numpy.log(C) * IR * IR * pow(10, 6) * CURRF * CURRF * DZR
    Lz = pow(10 , -9) * ZR * 2 * numpy.log(C)
    #Einductance is total inductive energy (in all inductances)
    Einductance = 0.5 * pow(10, 6) * IR * IR * (CURRF * CURRF * Lz + L0)
    ni = massf * N0 * ((gamma + 1) / (gamma - 1))
    nimax = N0 * ((gamma + 1) / (gamma - 1))
    #Write out step-by-step integration
    step = [TR , IR , VR, ZR, ZZR, TR, IR * CURRF, Lz * pow(10,-9), 100 * Einductance / (0.5 * C0 * V0 * V0), 100 * EINP / (0.5 * C0 * V0 * V0), ni / (10 ^ 23), TM / pow(10, 6), nimax / pow(10 , 23), 100 * Wpiston / (0.5 * C0 * V0 * V0)]
    axialphase.append(step)
    V = z * II + ZZ * I
    V = V * BE
    #Compute Generating Quantities (ie acceleration and IDOT) before loopback to continue integration
    AC = (AL * AL * I * I - ZZ * ZZ) / z
    II = (1 - IO - BE * ZZ * I - RESF * I) / (1 + BE * z)
    #Check end condition reached
    if z >= 1:
        break

#prep for next phase
Laxial = ZR * 2 * numpy.log(C)
ZG = ZZ
CFA = CURRF
#Introduce differential in current factors for axial and radial phases
CurrentFactorRatio = currfr / CURRF
CFR = CurrentFactorRatio
BE = BE * CFR
BF = BF * CFR
CURRF = CURRF * CFR
AAg = AA / numpy.sqrt(gamma + 1)
#End of Axial Phase
#print(axialphase[0])

[0.0036331804249169902, 0.4954336943068623, 0.0, 7.097468791980212e-05, 0.019535139910213443, 0.0036331804249169902, 0.3468035860148036, 1.7238958258231653e-22, 0.0004000003071668927, 3.071668926012185e-10, 2.7339280417935474e+21, 0.0, 9.91048915150161, 3.0716689260121857e-10]


In [16]:
#Start of Radial Inward Shock Phase
# Radial phase RI, distances are relative to radius a.
# KS is shock position, KP is radial piston position, ZF is focus
# pinch length, all normalized to inner radius a; VS and VP are
# radial shock and piston speed,VZ is axial pinch length elongating rate
# Distances, radius and speeds are relative to radius of anode.
# AS BEFORE QUANTITIES WITH AN R ATTACHED HAVE BEEN RE-COMPUTED AS REAL, i.e. UN-NORMALIZED QUANTITIES EXPRESSED IN USUAL LABORATORY UNITS.

# FOR THIS PHASE Z=EFFECTIVE CHARGE NUMBER!!!

#init the shock array
radialshock = []

